In [163]:
import numpy as np
import numpy.linalg as LA
from numpy.polynomial import Chebyshev as T
from scipy import optimize
from maxvolpy.maxvol import rect_maxvol, maxvol
import matplotlib.pyplot as plt
import gen_mat as gen
%matplotlib inline

In [164]:
def grad(points):
    """
    Analytical gradient 
    INPUT
        points – vector of points
    OUTPUT
        grad – gradient of log(det(A.T*A)) in points
    """
    points = np.stack(np.split(points, num_of_dim),1)
    idx = gen.indeces_K_cut(num_of_dim, num_col)
    max_degree = np.max(idx)
    
    T_col_prod = np.ones((points.shape[0], len(idx)), dtype = np.float64)
    T_deriv = np.zeros((points.shape[0]*num_of_dim, max_degree + 1), dtype = np.float64) 
    T_val = np.zeros((points.shape[0]*num_of_dim, max_degree + 1), dtype = np.float64) 
    for i in range(len(idx)):
        for l in range(len(idx[i])):
            T_col_prod[:, i] = np.multiply(T_col_prod[:, i], T.basis(idx[i][l])(points[:,l]))  
    for i in range(max_degree + 1):
        T_deriv[:, i] = T.deriv(T.basis(i))(points.T.ravel()[:])
        T_val[:, i] = T.basis(i)(points.T.ravel()[:])
    
    grad = np.zeros(points.shape[0]*num_of_dim, dtype = np.float64)    
    A = np.split(gen.GenMat(num_col, points, poly = gen.cheb, poly_diff = gen.cheb_diff), num_of_dim + 1)[0]
    B_inv = LA.inv(np.dot(A.conj().T, A))
    for k in range(len(grad)):
        for i in range(B_inv.shape[0]):
            for j in range(B_inv.shape[0]):
                col = k//points.shape[0]
                row = k%points.shape[0]
                grad[k] += B_inv[j,i] * (T_col_prod[row,i]*T_col_prod[row,j]*(T_deriv[k,idx[i][col]]/T_val[k,idx[i][col]] + T_deriv[k,idx[j][col]]/T_val[k,idx[j][col]]))
    return -grad
                
def loss_func(points):
    points = np.stack(np.split(points.T.ravel(),num_of_dim),1)
    A = np.split(gen.GenMat(num_col, points, poly = gen.cheb, poly_diff = gen.cheb_diff), num_of_dim + 1)[0]
    S = LA.svd(A, compute_uv = False)
    ld = 2.0*np.sum(np.log(S))
    return ld

In [165]:
# Workspace
num_col = 14
num_of_points = 14
num_of_dim = 2
assert num_col <= num_of_points, 'num_of_points < num_col'
l_bound = -2
u_bound = 2
p = l_bound + (u_bound - l_bound)*np.random.rand(num_of_points, num_of_dim)
x_0 = p.T.ravel()
bnds = tuple((l_bound, u_bound) for i in np.zeros(num_of_dim*num_of_points))

def f(points):
    #return 2*np.exp(-((points[:,0]**2)/2. + (points[:,1]**2)/2. + (points[:,2]**2)/2.))
    return (1 - points[:,0])**2 + 100*(points[:,1] - points[:,0]**2)**2
    #return (np.sin((points[:,0]**2)/2. - (points[:,1]**2)/4. + 3) * np.cos(2*points[:,0] + 1 - np.exp(points[:,1]))) 

print('Max degree = ', np.max(gen.indeces_K_cut(num_of_dim, num_col)))

#Rosenbrock function 
#f = (1 - P[:,0])**2 + 100*(P[:,1] - P[:,0]**2)**2

#Gaussian
#f = 2*np.exp(-((P[:,0]**2)/2. + (P[:,1]**2)/2.))

#f = (1 - P[:,0]**2)*P[:,1] + P[:,1]**2

Max degree =  4


In [166]:
# BFGS
%time res = optimize.fmin_l_bfgs_b(lambda x: -loss_func(x), x_0, fprime = grad, factr = 10.0, bounds = bnds)

CPU times: user 2.04 s, sys: 35.1 ms, total: 2.07 s
Wall time: 2.07 s


In [167]:
# Check of the correctness of gradient calculation
optimize.check_grad(lambda x: -loss_func(x), grad, x_0)

1.9020477184287168e-05

In [168]:
# Approximation error

M = np.split(gen.GenMat(num_col, np.stack(np.split(res[0].T.ravel(),num_of_dim),1), poly = gen.cheb, poly_diff = gen.cheb_diff), num_of_dim + 1)[0]

c = LA.solve(np.dot(M.conj().T, M), np.dot(M.conj().T, f(np.stack(np.split(res[0].T.ravel(),num_of_dim),1))))

test = l_bound + (u_bound - l_bound)*np.random.rand(10000, num_of_dim)

M_new = np.split(gen.GenMat(num_col, np.stack(np.split(test.T.ravel(),num_of_dim),1), poly = gen.cheb, poly_diff = gen.cheb_diff), num_of_dim + 1)[0]

print("Error = ", LA.norm(f(test) - np.dot(M_new, c),2) / LA.norm(f(test),2), "\n")

Error =  3.30495903957e-15 

